# Experiment for neural style transfer using AlexNet and VGG16 (MaxPool)

# Main Function for the experiment

In [ ]:
import torch 
from torchvision import transforms , models 
from PIL import Image 
import matplotlib.pyplot as plt
import numpy as np

device = ("cuda" if torch.cuda.is_available() else "cpu") # use gpu if available

from google.colab import drive
drive.mount('/content/drive')

# Grace Directory
# ! cp -r 'drive/MyDrive/ds301HW/neural style transfer' '/content'

# Stephen Directory
# ! cp -r 'drive/MyDrive/DS-UA 301/DS301' '/content' 

# Feature Extraction on each layer
def model_activations(input,model): 
  global layers

  features = {}
  x = input
  x = x.unsqueeze(0)
  for name,layer in model._modules.items():
      x = layer(x)
      if name in layers:
          features[layers[name]] = x 
  return features

  # Preprocess image
def imcnvt(image):
    x = image.to("cpu").clone().detach().numpy().squeeze()
    x = x.transpose(1,2,0)
    x = x*np.array((0.5,0.5,0.5)) + np.array((0.5,0.5,0.5))
    return np.clip(x,0,1)

# Helper function on sytle loss
def gram_matrix(imgfeature):
    _,d,h,w = imgfeature.size()
    imgfeature = imgfeature.view(d,h*w)
    gram_mat = torch.mm(imgfeature,imgfeature.t())
    return gram_mat

# Load and preprocess and plot the image
def preprocess_and_plot(plot = True):
  global content_path, style_path
  transform = transforms.Compose([transforms.Resize(300),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

  content = Image.open(content_path).convert("RGB")
  content = transform(content).to(device)
  print("Content shape => ", content.shape)
  style = Image.open(style_path).convert("RGB")
  style = transform(style).to(device)

  if plot:
    fig, (ax1,ax2) = plt.subplots(1,2)
    ax1.imshow(imcnvt(content),label = "Content")
    plt.axis('off')
    ax2.imshow(imcnvt(style),label = "Style")
    plt.axis('off')
    plt.show()
  target = content.clone().requires_grad_(True).to(device)
  return content, style, target

# Feature Extraction for both sytle and content loss
def extract_features(content, style):
  global model, layers
  style_features = model_activations(style,model)
  content_features = model_activations(content,model)
  style_grams = {layer:gram_matrix(style_features[layer]) for layer in style_features}
  return content_features, style_features, style_grams

# Trianing loop for the experiment
def training_loop(content_features, target, style_grams, epochs, print_after, aggregate = True):

  global content_index, style_index
  optimizer = torch.optim.Adam([target],lr=0.007)

  for i in range(1,epochs+1):

    target_features = model_activations(target,model)

    if aggregate:

      content_loss = 0
      for layer in content_features.keys():
        content_loss = torch.mean((content_features[layer]-target_features[layer])**2)
        content_loss += content_loss

      content_loss = content_loss.mean()

    else:
      
      content_loss = torch.mean((content_features[list(content_features.keys())[-2]]-target_features[list(target_features.keys())[-2]])**2) 


    style_loss = 0
    for layer in style_wt_meas:

        style_gram = style_grams[layer]
        target_gram = target_features[layer]
        _,d,w,h = target_gram.shape
        target_gram = gram_matrix(target_gram)
        style_loss += (style_wt_meas[layer]*torch.mean((target_gram-style_gram)**2))/d*w*h
    
    total_loss = content_wt*content_loss + style_wt*style_loss
    
    if i%1000==0:       
        print("epoch ",i," ", total_loss)
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    if i%print_after == 0:
        plt.imshow(imcnvt(target),label="Epoch "+str(i))
        plt.axis('off')
        plt.show()
        plt.imsave(f'vgg{content_index}-{style_index}.png',imcnvt(target),format='png')

# Main function for the experiment
def main(content_wt, epoch,print_after, plot= False, aggregate = True):
  content, style, target = preprocess_and_plot(plot = plot)
  content_features, style_features, style_grams = extract_features(content, style)
  training_loop(content_features, target, style_grams, epochs = epoch, print_after= print_after, aggregate = aggregate)

# Images for the experiment

In [ ]:
# Image paths for the experiment
content_index, style_index = 3, 5 # Change the index to change the image

# Grace Directory
# content_path = f"neural style transfer/images/{content_index}.jpeg"
# style_path = f"neural style transfer/style/{style_index}.jpeg"

# Stephen Directory
# content_path = f"DS301/images/{content_index}.jpeg"
# style_path = f"DS301/style/{style_index}.jpeg"

# Experiments for the neural style transfer using VGG19

In [ ]:
# Experiments for the VGG19 model

def selectLayers(layerALL = True):

    if layerALL == False:

        # Selecting the layers for the experiment
        layers = {
            '4': 'maxpool_4',
            '18': 'maxpool_18',
            '36': 'maxpool_36'
        }
        # Selecting the weights for the experiment
        style_wt_meas = {
                    "maxpool_4": 1.0,
                    "maxpool_18": 0.4,
                    "maxpool_36": 0.1
                }

    else:

        # Selecting the layers for the experiment
        layers = {
            '4': 'maxpool_4',
            '9': 'maxpool_9',
            '18': 'maxpool_18',
            '27': 'maxpool_27',
            '36': 'maxpool_36'
        }
        # Selecting the weights for the experiment
        style_wt_meas = {
                    "maxpool_4": 1.0,
                    "maxpool_9": 0.8,
                    "maxpool_18": 0.4,
                    "maxpool_27": 0.2,
                    "maxpool_36": 0.1
                }

    return layers, style_wt_meas

# Whehter select all the layers for the experiment
layerALL = True
layers, style_wt_meas = selectLayers(layerALL = layerALL)

# Loading the pretrained VGG16 model
model = models.vgg19(pretrained= True).features

# Setting the content and style weights
content_wt = 100
style_wt = 1e4


for p in model.parameters():
    p.requires_grad = False
model.to(device)

content_wts = 100
style_wt = 10000
epoch = 100000
print_after = 1000


# Running the experiment for the VGG16 model
main(content_wt, epoch = epoch, print_after = print_after, plot = True, aggregate = True)

# Experiments for the neural style transfer using AlexNet

In [ ]:
# Experiments for the AlexNet model

def selectLayers(layerALL = True):

    if layerALL == False:

        # Selecting the layers for the experiment
        layers = {
            '2' : 'maxpool_2',
            '12': 'maxpool_12'
        }
        # Selecting the weights for the experiment
        style_wt_meas = {
                    "maxpool_2": 1.0,
                    "maxpool_12": 0.1
                }

    else:

        # Selecting the layers for the experiment
        layers = {
            '2' : 'maxpool_2',
            '5': 'maxpool_5',
            '12': 'maxpool_12'
        }
        # Selecting the weights for the experiment
        style_wt_meas = {
                    "maxpool_2": 1.0,
                    "maxpool_5": 0.5,
                    "maxpool_12": 0.1
                }

    return layers, style_wt_meas

# Whehter select all the layers for the experiment
layerALL = True
layers, style_wt_meas = selectLayers(layerALL = layerALL)

# Loading the pretrained VGG16 model
model = models.alexnet(pretrained= True).features

# Setting the content and style weights
content_wt = 100
style_wt = 1e4


for p in model.parameters():
    p.requires_grad = False
model.to(device)

content_wts = 100
style_wt = 10000
epoch = 100000
print_after = 1000


# Running the experiment for the VGG16 model
main(content_wt, epoch = epoch, print_after = print_after, plot = True, aggregate = True)